# Gender Bias Detection in Image Captions

This notebook demonstrates how to use our fine-tuned LlamaV-o1 model to detect gender bias in image-caption pairs. The model has been trained to:
1. Analyze an image and its caption
2. Generate a step-by-step chain-of-thought reasoning
3. Provide a final judgment (Biased or Not Biased)

Let's get started!

## 1. Setup and Installation

First, we'll install the required packages:

In [ ]:
!pip install torch transformers accelerate pillow matplotlib sentencepiece

## 2. Load the Model

Now we'll load our fine-tuned LlamaV-o1 model. You have two options:
1. Load from a local path (if you have the model files)
2. Load from Hugging Face Hub (if the model is published there)

Let's implement both options:

In [ ]:
import os
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, CLIPProcessor
from PIL import Image
import matplotlib.pyplot as plt
import requests
from io import BytesIO

# Choose which option to use
use_local_model = False  # Set to True to use local model files

# Option 1: Load from local path
if use_local_model:
    model_path = "path/to/fine_tuned_model"  # Change this to your model path
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModelForCausalLM.from_pretrained(
        model_path,
        torch_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32,
        device_map="auto" if torch.cuda.is_available() else None,
    )
    # Also load the image processor
    image_processor = CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14")
    
# Option 2: Load from Hugging Face Hub
else:
    # Replace with your actual model ID if published
    model_id = "your-username/llamav-o1-gender-bias-detector"
    
    # For this demo, we'll use the base LlamaV-o1 model (not fine-tuned)
    # In a real scenario, you would use your fine-tuned model
    model_id = "mbzuai-oryx/LlamaV-o1-7B"
    
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        torch_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32,
        device_map="auto" if torch.cuda.is_available() else None,
    )
    # Load the image processor
    image_processor = CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14")

# Ensure the tokenizer has a padding token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print(f"Model loaded successfully!")

## 3. Define Inference Function

Now we'll define a function to run inference on image-caption pairs:

In [ ]:
def analyze_bias(image, caption, max_new_tokens=512):
    """Analyze gender bias in an image-caption pair.
    
    Args:
        image: PIL Image or path to image
        caption: str, the caption to analyze
        max_new_tokens: int, maximum number of tokens to generate
        
    Returns:
        dict with reasoning and bias label
    """
    # Load the image if it's a path
    if isinstance(image, str):
        if image.startswith("http"):
            response = requests.get(image)
            image = Image.open(BytesIO(response.content)).convert("RGB")
        else:
            image = Image.open(image).convert("RGB")
    
    # Process the image
    pixel_values = image_processor(image, return_tensors="pt").pixel_values
    if torch.cuda.is_available():
        pixel_values = pixel_values.cuda()
    
    # Prepare the prompt
    prompt = f"Analyze this image and caption for gender bias: \"{caption}\""
    
    # Tokenize the prompt
    inputs = tokenizer(prompt, return_tensors="pt")
    if torch.cuda.is_available():
        inputs = {k: v.cuda() for k, v in inputs.items()}
    
    # Run inference
    # Note: This is a simplified inference approach
    # For LlamaV-o1, you would need to modify this to match the model's specific input format
    # including how to pass the image features
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            num_beams=1,
        )
    
    # Decode the output
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Extract the reasoning and bias label
    result_text = generated_text[len(prompt):].strip()
    
    # Parse the result to extract reasoning and bias label
    # This is a simplified approach - in a real scenario, you might need more robust parsing
    if "Biased" in result_text.split("\n")[-1]:
        bias_label = "Biased"
    elif "Not Biased" in result_text.split("\n")[-1]:
        bias_label = "Not Biased"
    else:
        # Default if we can't clearly determine
        bias_label = "Unclear"
    
    reasoning = result_text
    
    return {
        "reasoning": reasoning,
        "bias_label": bias_label
    }

## 4. Example 1: Potentially Biased Caption

Let's analyze a potentially biased image-caption pair:

In [ ]:
# Example 1: Potentially biased caption
image_url_1 = "https://example.com/path/to/image1.jpg"  # Replace with a real image URL
caption_1 = "A woman in the kitchen preparing dinner while her husband relaxes in the living room."

# For demonstration purposes with a real image URL
# (replace with your own example URL)
image_url_1 = "https://images.unsplash.com/photo-1556911220-bff31c812dba"

# Display the image
image_1 = Image.open(BytesIO(requests.get(image_url_1).content)).convert("RGB")
plt.figure(figsize=(10, 8))
plt.imshow(image_1)
plt.axis('off')
plt.title(f"Caption: {caption_1}")
plt.show()

# Analyze for bias
result_1 = analyze_bias(image_1, caption_1)

# Display results
print("✨ Analysis Results ✨")
print(f"Caption: {caption_1}")
print(f"\nBias Label: {result_1['bias_label']}")
print("\nReasoning:")
print(result_1['reasoning'])

## 5. Example 2: Non-Biased Caption

Now let's analyze a likely non-biased image-caption pair:

In [ ]:
# Example 2: Non-biased caption
image_url_2 = "https://example.com/path/to/image2.jpg"  # Replace with a real image URL
caption_2 = "Two scientists working together in a laboratory on a chemical experiment."

# For demonstration purposes with a real image URL
# (replace with your own example URL)
image_url_2 = "https://images.unsplash.com/photo-1532094349884-543bc11b234d"

# Display the image
image_2 = Image.open(BytesIO(requests.get(image_url_2).content)).convert("RGB")
plt.figure(figsize=(10, 8))
plt.imshow(image_2)
plt.axis('off')
plt.title(f"Caption: {caption_2}")
plt.show()

# Analyze for bias
result_2 = analyze_bias(image_2, caption_2)

# Display results
print("✨ Analysis Results ✨")
print(f"Caption: {caption_2}")
print(f"\nBias Label: {result_2['bias_label']}")
print("\nReasoning:")
print(result_2['reasoning'])

## 6. Upload Your Own Image and Caption

Now you can try with your own image and caption:

In [ ]:
# Upload an image
from google.colab import files
uploaded = files.upload()

# Get the filename of the uploaded image
uploaded_filename = list(uploaded.keys())[0]
user_image = Image.open(uploaded_filename).convert("RGB")

# Display the image
plt.figure(figsize=(10, 8))
plt.imshow(user_image)
plt.axis('off')
plt.show()

In [ ]:
# Enter a caption
user_caption = input("Enter a caption for the image: ")

# Analyze for bias
user_result = analyze_bias(user_image, user_caption)

# Display results
print("\n✨ Analysis Results ✨")
print(f"Caption: {user_caption}")
print(f"\nBias Label: {user_result['bias_label']}")
print("\nReasoning:")
print(user_result['reasoning'])

## 7. Conclusion

In this notebook, we demonstrated how to use our fine-tuned LlamaV-o1 model to analyze image-caption pairs for gender bias. The model provides:

1. A chain-of-thought reasoning process that examines various aspects of potential bias
2. A final judgment (Biased, Not Biased, or Unclear)

This approach can help identify and address gender bias in image descriptions, contributing to more inclusive and fair visual content.

### Limitations

Note that this model has several limitations:
- It's trained on a specific dataset and may not generalize to all types of bias
- The judgment is based on the fine-tuning data, which reflects certain perspectives
- The model may sometimes produce unclear or inconsistent reasoning

Always use human judgment alongside the model's analysis for important decisions.